The following cell sets the input values required to run the the optimisation model ``..\MINIZINC\hydrogen_plant.mzn``.

This input data are stored in ``..\MINIZINC\hydrogen_plant_data.dzn``. This input file preparation is carried out by ``make_dzn_file`` function that is in ``..\PYTHON\PACKAGE\optimisation.py``.

Then the minizinc optimisation is executed by ``optimise()`` function that is in ``..\PYTHON\PACKAGE\optimisation.py``. 

``projdirs`` stores the key paths for the entire package.

# Set the location and update the solar and wind data

In [ ]:
import pandas as pd
import numpy as np
import os
from projdirs import datadir #load the path that contains the data files 
from PACKAGE.optimisation import Optimise
from PACKAGE.component_model import pv_gen, wind_gen, SolarResource_solcast, WindSource_solcast

#Locations covered
Newman = [-23.35, 119.75]
Sydney = [-33.856784, 151.215297]
Tom_Price = [-22.69, 117.79]
Port_Augusta = [-32.49, 137.77]
Whyalla = [-33.04, 137.59]
Pinjarra = [-32.63, 115.87]
Gladstone = [-23.84, 151.25]
Burnie = [-41.05, 145.91]

#Choose the location
Location = 'Newman'

#Update the weather data files
weather_data_file = 'weather_data_%s.csv'%(Location)
SolarResource_solcast(weather_data_file)
WindSource_solcast(weather_data_file)

In [ ]:
# Choose the underground storage type
storage_type = 'Lined Rock'  

# set the load value
load = 5 # kgH2/s

# create a dictionary that contains the inputs for optimisation.
#these inputs are used by make_dzn_file function to create an input text file called hydrogen_plant_data.dzn.                 

simparams = dict(EL_ETA = 0.70,       #efficiency of electrolyser
                 BAT_ETA_in = 0.95,   #charging efficiency of battery
                 BAT_ETA_out = 0.95,  #discharg efficiency of battery
                 C_PV = 1122,          #[USD/kW] unit cost of PV
                 C_WIND = 1455,           #[USD/kW] unit cost of Wind
                 C_EL = 1067,          #[USD/W] unit cost of electrolyser
                 UG_STORAGE_CAPA_MAX = 1e10,   #maximum available salt caevern size (kg of H2)
                 C_PIPE_STORAGE = 516, #unit cost of line packing (USD/kg of H2)
                 PIPE_STORAGE_CAPA_MIN = 100e3, #minimum size of linepacking (kg of H2)
                 C_BAT_ENERGY = 196,        #[USD/kWh] unit cost of battery energy storage
                 C_BAT_POWER = 405,        #[USD/kW] unit cost of battery power capacpity
                 )       

#results = Optimise(load, CF, storage_type, simparams)



import multiprocessing as mp

pool = mp.Pool(mp.cpu_count()-1)
print('Started!')
output = [pool.apply_async(Optimise,
                           args=(load, CF, storage_type, params))
          for load in [5]
          for CF in [50,60,70,80,90,100]
          for storage_type in ['Lined Rock'] 
          for params in [simparams]]

pool.close()
pool.join()
print('Completed!')


In [ ]:
RESULTS = pd.DataFrame(columns=['cf','capex[USD]','pv_capacity[kW]',
                                    'wind_capacity[kW]','el_capacity[kW]',
                                    'ug_capcaity[kgH2]','pipe_storage_capacity[kgH2]',
                                    'bat_e_capacity[kWh]','bat_p_capacity[kW]',
                                    'pv_cost[USD]', 'wind_cost[USD]','el_cost[USD]',
                                    'ug_storage_cost[USD]','pipe_storage_cost[USD]',
                                    'bat_cost[USD]', 'load[kg/s]'])
for i in output:
    results = i.get()
    RESULTS = RESULTS.append({'cf': results['CF'],
                        'capex[USD]': results['CAPEX'][0],
                        'pv_capacity[kW]': results['pv_max'][0],
                        'wind_capacity[kW]': results['wind_max'][0],
                        'el_capacity[kW]': results['el_max'][0],
                        'ug_capcaity[kgH2]': results['ug_storage_capa'][0],
                        'pipe_storage_capacity[kgH2]': results['pipe_storage_capa'][0],
                        'bat_e_capacity[kWh]': results['bat_e_capa'][0],
                        'bat_p_capacity[kW]': results['bat_p_max'][0],
                        'pv_cost[USD]': results['pv_max'][0]*simparams['C_PV'],
                        'wind_cost[USD]': results['wind_max'][0]*simparams['C_WIND'],
                        'el_cost[USD]': results['el_max'][0]*simparams['C_EL'],
                        'ug_storage_cost[USD]': results['ug_storage_capa'][0]*results['C_UG_STORAGE'],
                        'pipe_storage_cost[USD]':results['pipe_storage_capa'][0]*simparams['C_PIPE_STORAGE'],
                        'bat_cost[USD]': results['bat_p_max'][0]*simparams['C_BAT_ENERGY'],
                        'load[kg/s]':results['LOAD'][0] }, ignore_index=True)
    

In [ ]:
RESULTS

In [ ]:

RESULTS = RESULTS.append({'cf': simparams['CF'],
                        'capex[USD]': results['CAPEX'][0],
                        'pv_capacity[kW]': results['pv_max'][0],
                        'wind_capacity[kW]': results['wind_max'][0],
                        'el_capacity[kW]': results['el_max'][0],
                        'ug_capcaity[kgH2]': results['ug_storage_capa'][0],
                        'pipe_storage_capacity[kgH2]': results['pipe_storage_capa'][0],
                        'bat_e_capacity[kWh]': results['bat_e_capa'][0],
                        'bat_p_capacity[kW]': results['bat_p_max'][0],
                        'pv_cost[USD]': results['pv_max'][0]*simparams['C_PV'],
                        'wind_cost[USD]': results['wind_max'][0]*simparams['C_WIND'],
                        'el_cost[USD]': results['el_max'][0]*simparams['C_EL'],
                        'ug_storage_cost[USD]': results['ug_storage_capa'][0]*simparams['C_UG_STORAGE'],
                        'pipe_storage_cost[USD]':results['pipe_storage_capa'][0]*simparams['C_PIPE_STORAGE'],
                        'bat_cost[USD]': results['bat_p_max'][0]*simparams['C_BAT_ENERGY'],
                        'load[kg/s]':results['LOAD'][0] }, ignore_index=True)

In [ ]:
RESULTS = pd.DataFrame(columns=['cf','capex[USD]','pv_capacity[kW]',
                                    'wind_capacity[kW]','el_capacity[kW]',
                                    'ug_capcaity[kgH2]','pipe_storage_capacity[kgH2]',
                                    'bat_e_capacity[kWh]','bat_p_capacity[kW]',
                                    'pv_cost[USD]', 'wind_cost[USD]','el_cost[USD]',
                                    'ug_storage_cost[USD]','pipe_storage_cost[USD]',
                                    'bat_cost[USD]', 'load[kg/s]'])

RESULTS = RESULTS.append({'cf': simparams['CF'],
                                  'capex[USD]': results['CAPEX'][0],
                                  'pv_capacity[kW]': results['pv_max'][0],
                                  'wind_capacity[kW]': results['wind_max'][0],
                                  'el_capacity[kW]': results['el_max'][0],
                                  'ug_capcaity[kgH2]': results['ug_storage_capa'][0],
                                  'pipe_storage_capacity[kgH2]': results['pipe_storage_capa'][0],
                                  'bat_e_capacity[kWh]': results['bat_e_capa'][0],
                                  'bat_p_capacity[kW]': results['bat_p_max'][0],
                                  'pv_cost[USD]': results['pv_max'][0]*simparams['C_PV'],
                                  'wind_cost[USD]': results['wind_max'][0]*simparams['C_WIND'],
                                  'el_cost[USD]': results['el_max'][0]*simparams['C_EL'],
                                  'ug_storage_cost[USD]': results['ug_storage_capa'][0]*simparams['C_UG_STORAGE'],
                                  'pipe_storage_cost[USD]':results['pipe_storage_capa'][0]*simparams['C_PIPE_STORAGE'],
                                  'bat_cost[USD]': results['bat_p_max'][0]*simparams['C_BAT_ENERGY'],
                                  'load[kg/s]':results['LOAD'][0] }, ignore_index=True)


### Post process data

In [ ]:
path = r'C:\Nextcloud\HILT-CRC---Green-Hydrogen\DATA\SAM INPUTS\WEATHER_DATA'
weather_data = pd.read_csv(path + '\weather_data_%s.csv'%(Location),skiprows=2)

def read_data_for_plotting(results):
    data_to_plot = {}
    (KEYS,VALUES) = (list(results.keys()),list(results.values()))
    for (k,v) in zip(KEYS,VALUES):
        if len(v)>1:
            data_to_plot[k]=v[0:int(results['N'][0])]
    return(pd.DataFrame(data_to_plot))


data_plot = read_data_for_plotting(results)

beam = weather_data['DNI'].tolist()
G_H = weather_data['GHI']
wind_speed = weather_data['Wind Speed']

water_rate = 15 #L/kgH2  range: 15-20 L/kgH2
c_water = 5 # $/m3

data_plot['beam'] = beam
data_plot['t'] = np.arange(0,results['N'])
data_plot['time'] =pd.to_datetime('2021-01-01') + pd.to_timedelta(data_plot.t, 'h')
data_plot.drop('t',axis=1,inplace=True)
data_plot['G_H'] = G_H
data_plot['wind_speed'] = wind_speed

# water_mdot = results['he_out']*3.6*water_rate,  #T of H2O/hr

## Plot solar irradiance and wind speed

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=list(data_plot.time), y=list(data_plot.beam),
              line_shape='hv', name='Direct Solar Irrad. [W/m<sup>2</sup>]',
               line_color = 'red', opacity= 0.5, yaxis="y1"),
                )

fig.add_trace(
    go.Scatter(x=list(data_plot.time), y=list(data_plot.G_H),
              line_shape='hv', name='Global H. Solar Irrad. [W/m<sup>2</sup>]',
               line_color = 'orange', opacity= 0.5, yaxis="y1"),
                )

fig.add_trace(
    go.Scatter(x=list(data_plot.time), y=list(data_plot.wind_speed),
              line_shape='hv', name='Wind Speed [m/s]',
               line_color = 'green', opacity= 0.5, yaxis="y2"),
                )

fig.update_yaxes(dict(title = 'W/m<sup>2</sup>'),
                 linecolor='black',
                 mirror=True, secondary_y=False)

fig.update_yaxes(dict(title = 'm/s'), secondary_y=True)

fig.update_xaxes(linecolor='black',mirror=True)


fig.update_layout(width=900, height=300,
                  margin=dict(l=0, r=0, t=20, b=0))
path = r'C:\Users\Ahmad Mojiri\OneDrive - Australian National University\HILT CRC\Hydrogen supply HILTCRC\Pictures'
# fig.write_image(path + r'\PVW-CF=%s.png'%str(simparams['CF']))

fig.show()

### Plot mass flow

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure
# fig = go.Figure()
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=list(data_plot.time), y=list(data_plot.ug_storage_level),
              name='ug_storage_level [kg]',yaxis="y1"),
                )
fig.add_trace(
    go.Scatter(x=list(data_plot.time), y=list(data_plot.pipe_storage_level),
              name='pipe_storage_level [kg]',yaxis="y1"),
                )               

fig.add_trace(
    go.Scatter(x=list(data_plot.time), y=list(data_plot.comp1_hflow),
              line_shape='hv',name='H<sub>2</sub> to compressor 1 [kg/s]',
               yaxis="y2", opacity=0.5)
                )

fig.add_trace(
    go.Scatter(x=list(data_plot.time), y=list(data_plot.comp2_hflow),
              line_shape='hv',name='H<sub>2</sub> to comrpessor 2 [kg/s]',
               yaxis="y2", opacity=0.5)
                )

fig.add_trace(
    go.Scatter(x=list(data_plot.time), y=list(data_plot.pipe_storage_hout),
              line_shape='hv',name='H<sub>2</sub> from pipe [kg/s]',yaxis="y2")
                )

fig.add_trace(
    go.Scatter(x=list(data_plot.time), y=list(data_plot.ug_storage_hout),
              line_shape='hv',name='H<sub>2</sub> from UG [kg/s]',yaxis="y2")
                )

fig.add_trace(
    go.Scatter(x=list(data_plot.time), y=list(data_plot.res_hout),
              line_shape='hv',name='Unserved Load [kg/s]',yaxis="y2")
                )
# fig.add_trace(
#     go.Scatter(x=list(data_plot.time), y=list(data_plot.water_mdot),
#               line_shape='hv',name='Water to Electrolyser [kg/s]',
#                opacity = 0.5, yaxis="y2")
#                 )

fig.update_yaxes(dict(title = '<b>kg of H<sub>2</sub></b>',linecolor='black',
                          mirror=True ), secondary_y=False,)

fig.update_yaxes(dict(title = '<b>kg/s</b>',linecolor='black',
                          mirror=True,), secondary_y=True)

fig.update_xaxes(dict(linecolor='black',
                          mirror=True))

fig.add_annotation(text='<b>Capacity Factor = %s</b>'%str(simparams['CF']),
                   xref="paper", yref="paper",
                   align='center',
                   x=0.5, y=0.95, showarrow=False)

fig.update_layout(width=900, height=300,
                  margin=dict(l=0, r=0, t=30, b=0))
path = r'C:\Users\Ahmad Mojiri\OneDrive - Australian National University\HILT CRC\Hydrogen supply HILTCRC\Pictures'
# fig.write_image(path + r'\Hflow-CF=%s.png'%str(simparams['CF']))
fig.show()

In [ ]:
path_to_file = r'C:\Nextcloud\HILT-CRC---Green-Hydrogen\DATA\OPT_OUTPUTS'
result_file = r'\H2data(%s-Salt cavern).csv'%Location 
# existing_results = pd.read_csv(path_to_file+result_file)
# existing_results.append(RESULTS,ignore_index=True)
data_plot.to_csv(path_to_file+result_file, index=False)

### Plot power flow

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# Create figure
# fig = go.Figure()
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=list(data_plot.time), y=list(data_plot.pv_pout),
              line_shape='hv', name='PV Generation [kW]',
              line_color = 'orange', opacity= 0.5),
                )
fig.add_trace(
    go.Scatter(x=list(data_plot.time), y=list(data_plot.wind_pout),
              line_shape='hv', name='Wind Generation [kW]',
              line_color = 'green', opacity = 0.5),
                )
fig.add_trace(
    go.Scatter(x=list(data_plot.time), y=list(data_plot.curtail_p),
              line_shape='hv', name='Curtailed Power [kW]',
              line_color = 'grey', opacity = 0.5),
                )
fig.add_trace(
    go.Scatter(x=list(data_plot.time), y=list(data_plot.bat_pin),
              line_shape='hv', name='Battery Charging [kW]'),
                )
fig.add_trace(
    go.Scatter(x=list(data_plot.time), y=list(data_plot.bat_pout),
              line_shape='hv', name='Battery Discharging [kW]'),
                )
fig.add_trace(
    go.Scatter(x=list(data_plot.time), y=list(data_plot.el_pin),
              line_shape='hv', name='Electrolyer Pin [kW]', yaxis="y1"),
                )
fig.add_trace(
    go.Scatter(x=list(data_plot.time), y=list(data_plot.comp1_pin),
              line_shape='hv', name='Compressor1 Pin [kW]',
              line_color = 'orange', opacity= 0.5),
                )
fig.add_trace(
    go.Scatter(x=list(data_plot.time), y=list(data_plot.comp2_pin),
              line_shape='hv', name='Compressor2 Pin [kW]',
              line_color = 'orange', opacity= 0.5),
                )

fig.add_trace(
    go.Scatter(x=list(data_plot.time), y=list(data_plot.bat_e),
              line_shape='hv', name='Battery Energy [kWh]', yaxis="y2"),
                )


fig.update_yaxes(dict(title = 'kW',linecolor='black',
                          mirror=True ), secondary_y=False,)

fig.update_yaxes(dict(title = 'kWh',linecolor='black',
                          mirror=True,), secondary_y=True)

fig.update_xaxes(dict(linecolor='black',
                          mirror=True))

fig.add_annotation(text='<b>Capacity Factor = %s</b>'%str(simparams['CF']),
                   xref="paper", yref="paper",
                   align='center',
                   x=0.5, y=0.95, showarrow=False)

fig.update_layout(width=900, height=300,
                  margin=dict(l=0, r=0, t=20, b=0))
path = r'C:\Users\Ahmad Mojiri\OneDrive - Australian National University\HILT CRC\Hydrogen supply HILTCRC\Pictures'
# fig.write_image(path + r'\Pflow-CF=%s.png'%str(simparams['CF']))

fig.show()